# Sentinel-2 (global) <img align="right" src="../../resources/easi_logo.jpg">

#### Index
- [Overview](#Overview)
- [Setup (dask, imports, query)](#Setup)
- [Product definition (measurements, flags)](#Product-definition)
- [Quality layer (mask)](#Quality-layer)
- [Scaling and nodata](#Scaling-and-nodata)
- [Visualisation](#Visualisation)
- [Optional: Reproject](#Optional:-Reproject)
- [Appendix](#Appendix)

## Overview

Sentinel-2 is an Earth observation mission from the EU Copernicus Programme that systematically acquires optical imagery at high spatial resolution (up to 10 m for some bands). The mission is a constellation of two identical satellites in the same orbit, 180° apart for optimal coverage and data delivery. Together, they cover all Earth's land surfaces, large islands, inland and coastal waters every 3-5 days.

Sentinel-2A was launched on 23 June 2015 and Sentinel-2B followed on 7 March 2017.
Both of the Sentinel-2 satellites carry a wide swath high-resolution multispectral imager with 13 spectral bands.
For more information on the Sentinel-2 platforms and applications see the [European Space Agency website](http://www.esa.int/Applications/Observing_the_Earth/Copernicus/Overview4).

_Adapted from https://github.com/GeoscienceAustralia/dea-notebooks/blob/develop/DEA_datasets/Sentinel_2.ipynb_

#### Data source and documentation

EASI provides at least four Sentinel-2 products from three sources of data. These represent Analysis Ready Data (ARD) products corrected to surface reflectance.

We describe in this notebook the products that can be used in a global context. That is, products processed with globaly applicable algorithms.

| Name | Product | Information |
|------|---------|-------------|
| CopHub S2 Sen2cor | `cophub_s2_sr_sen2cor` | - Download S2 L2A sen2cor from EC Copernicus Data Hub ("CopHub"). Process to COG with the same structure as for `s2_l2a`. [More information below](#EC-Copernicus-Data-Hub)<br>- Use for global AOIs where custom preprocessing is required or analyses will benefit from a local data source |
| Sentinel-2 COGs | `s2_l2a` | - Direct indexing from Sentinel-2 Cloud-Optimized GeoTIFFs (uses the same S2 L2A Sen2cor source data as for `cophub_s2_sr_sen2cor`). [More information below](#Sentinel-2-Cloud-Optimized-GeoTIFFs)<br>- Use for global AOIs where Sen2cor surface correction is required |

See the companion notebook, "Sentinel-2 (Australia)" for an introduction to the Geoscience Australia products:

***---Split---***

| Name | Product | Information |
|------|---------|-------------|
| GA S2 NBAR | `ga_s2a_ard_nbar_granule`<br>`ga_s2b_ard_nbar_granule` | - Geoscience Australia S2 NBAR and NBART surface corrected for Australia. [More information below](#Geoscience-Australia-NBAR-corrected-for-Australia)<br>- Use for Australian AOIs |
| GA S2 NRT | `s2a_nrt_granule`<br>`s2b_nrt_granule` | - Geoscience Australia S2 Near-Real-Time NBAR and NBART surface corrected for Australia. [More information below](#Geoscience-Australia-NBAR-corrected-for-Australia)<br>- Use for Australian AOIs where analyses benefit from near real time data |
| | `s2(a,b)_nrt_granule` | Not currently indexed by EASI. They are currently available on NCI only |

#### EASI pipeline

| Task | `cophub_s2_sr_sen2cor` | `s2_l2a` |
|------|------------------------|----------|
| Source | https://scihub.copernicus.eu | https://registry.opendata.aws/sentinel-2-l2a-cogs |
| Download | Y | Index in place |
| Preprocess | Select bands by resolution | N/A |
| Format | Convert JP2 to COG | N/A |
| Prepare | Y | stac-to-dc |
| TODO | Update "band_0X" aliases<br>Capitalise SLC flag #9<br>Extend to L1+Atcor | |

***---Split---***

| Task | `ga_s2a_ard_nbar_granule` and `ga_s2b_ard_nbar_granule` |
|------|---------------------------------------------------------|
| Source | s3://dea-public-data/L2/sentinel-2-nbar/S2MSIARD_NBAR |
| Download | Index in place |
| Preprocess | N/A |
| Format | N/A |
| Prepare | s3-to-dc |
| TODO | |

## Setup

#### Dask

In [ ]:
from dask.distributed import Client

client = Client("tcp://10.0.90.144:44565")
client

#### Imports

In [ ]:
# Data tools
import numpy as np
import xarray as xr
import pandas as pd
from datetime import datetime

# Datacube
import datacube
from datacube.utils import masking  # https://github.com/opendatacube/datacube-core/blob/develop/datacube/utils/masking.py
from odc.algo import enum_to_bool   # https://github.com/opendatacube/odc-tools/blob/develop/libs/algo/odc/algo/_masking.py
from odc.algo import xr_reproject   # https://github.com/opendatacube/odc-tools/blob/develop/libs/algo/odc/algo/_warp.py
from datacube.utils.geometry import GeoBox, box  # https://github.com/opendatacube/datacube-core/blob/develop/datacube/utils/geometry/_base.py
from datacube.utils.rio import configure_s3_access

# Holoviews, Datashader and Bokeh
import hvplot.pandas
import hvplot.xarray
import holoviews as hv
import panel as pn
import colorcet as cc
import cartopy.crs as ccrs
from datashader import reductions
from holoviews import opts
# import geoviews as gv
# from holoviews.operation.datashader import rasterize
hv.extension('bokeh', logo=False)

# Python
import sys, os, re

# Optional EASI tools
sys.path.append(os.path.expanduser('~/hub-notebooks/scripts'))
import notebook_utils

#### ODC database

In [ ]:
dc = datacube.Datacube()

#### Example query

Change any of the parameters in the `query` object below to adjust the location, time, projection, or spatial resolution of the returned datasets.

Use the Explorer interface to check the temporal and spatial coverage for each product:
- https://explorer.csiro.easi-eo.solutions/cophub_s2_sr_sen2cor
- https://explorer.csiro.easi-eo.solutions/s2_l2a

***---Split---***

- https://explorer.csiro.easi-eo.solutions/ga_s2a_ard_nbar_granule and [ga_s2b_ard_nbar_granule](https://explorer.csiro.easi-eo.solutions/ga_s2b_ard_nbar_granule)
- https://explorer.csiro.easi-eo.solutions/s2a_nrt_granule and [s2b_nrt_granule](https://explorer.csiro.easi-eo.solutions/s2b_nrt_granule) 

Sentinel-2 datasets are stored with different coordinate reference systems (CRS), corresponding to multiple UTM zones used for S2 L1B tiling. S2 measurement bands also have different resolutions (10 m, 20 m and 60 m). As such S2 queries need to include the following two query parameters:

* `output_crs` - This sets a consistent CRS that all Sentinel-2 data will be reprojected to, irrespective of the UTM zone the individual image is stored in.
* `resolution` - This sets the resolution that all Sentinel-2 images will be resampled to. 

Use `mostcommon_crs()` to select a CRS. Adapted from https://github.com/GeoscienceAustralia/dea-notebooks/blob/develop/Tools/dea_tools/datahandling.py

In [ ]:
# Indonesia
min_longitude, max_longitude = (117.30, 120)
min_latitude, max_latitude = (-9.00, -7.50)
min_date = '2020-01-01'
max_date = '2020-06-01'
product = 'cophub_s2_sr_sen2cor'

query = {
    'product': product,                     # Product name
    'x': (min_longitude, max_longitude),    # "x" axis bounds
    'y': (min_latitude, max_latitude),      # "y" axis bounds
    'time': (min_date, max_date),           # Any parsable date strings
}

# Most common CRS
native_crs = notebook_utils.mostcommon_crs(dc, query)

query.update({
    'output_crs': native_crs,               # EPSG code
    'resolution': (-20, 20),                # Target resolution
    'group_by': 'solar_day',                # Scene ordering
    'dask_chunks': {'x': 2048, 'y': 2048},  # Dask chunks
})

# ***---Split---***

# Lake Hume for ga_s2a_ard_nbar_granule and s2a_nrt_granule
# min_longitude, max_longitude = (146.92, 147.25)
# min_latitude, max_latitude = (-36.25, -35.92)
# min_date = '2019-01-01'
# max_date = '2019-12-31'
# products = ['ga_s2a_ard_nbar_granule', 'ga_s2b_ard_nbar_granule']

# native_crs = ccrs.AlbersEqualArea()  # Target projection CRS

# query = {
#     'product': product,                          # Product name
#     'x': (min_longitude, max_longitude),    # "x" axis bounds
#     'y': (min_latitude, max_latitude),      # "y" axis bounds
#     'time': (min_date, max_date),           # Any parsable date strings
#     'output_crs': native_crs,              # EPSG code
#     'resolution': (-20, 20),                # Target resolution
#     'group_by': 'solar_day',                # Scene ordering
#     'dask_chunks': {'x': 2048, 'y': 2048},  # Dask chunks
# }

In [ ]:
# Optional. Some products require AWS S3 credentials to supplied

# S3 credentials - required for s2_l2a
# configure_s3_access(aws_unsigned=True,requester_pays=False,client=client)
# print('Configured s3 requester pays data access')

In [ ]:
# Load data
data = dc.load(**query)

notebook_utils.heading(notebook_utils.xarray_object_size(data))
display(data)

# Calculate valid (not nodata) masks for each layer
valid_mask = masking.valid_data_mask(data)
notebook_utils.heading('Valid data masks for each variable')
display(valid_mask)

## Product definition

Display the measurement definitions for the selected product.

Use `list_measurements` to show the details for a product, and `masking.describe_variable_flags` to show the flag definitions.

See [Appendix](#Appendix) for a Table of Sentinel-2 ARD bands and corresponding product measurement and alias names.

In [ ]:
# Measurement definitions for the selected product
measurement_info = dc.list_measurements().loc[query['product']]
notebook_utils.heading(f'Measurement table for product: {query["product"]}')
notebook_utils.display_table(measurement_info)

# Separate lists of measurement names and flag names
measurement_names = measurement_info[ pd.isnull(measurement_info.flags_definition)].index
flag_names        = measurement_info[pd.notnull(measurement_info.flags_definition)].index

notebook_utils.heading('Selected Measurement and Flag names')
notebook_utils.display_table(pd.DataFrame({
    'group': ['Measurement names', 'Flag names'],
    'names': [', '.join(measurement_names), ', '.join(flag_names)]
}))

# Flag definitions
for flag in flag_names:
    notebook_utils.heading(f'Flag definition table for flag name: {flag}')
    notebook_utils.display_table(masking.describe_variable_flags(data[flag]))

## Quality layer

In [ ]:
# Make L2_FLAGS image
flag_name = 'quality'
flag_data = data[[flag_name]].where(valid_mask[flag_name]).persist()   # Dataset
display(flag_data)

In [ ]:
# Make SCL image
# https://sentinel.esa.int/web/sentinel/technical-guides/sentinel-2-msi/level-2a/algorithm
# https://www.sentinel-hub.com/faq/how-get-s2a-scene-classification-sentinel-2/

from bokeh.models.tickers import FixedTicker

color_def = [
    (0,  '#000000', 'No data'),   # black
    (1,  '#ff0004', 'Saturated or defective'),   # red
    (2,  '#868686', 'Dark features or shadows'),   # gray
    (3,  '#774c0b', 'Cloud shadows'),   # brown
    (4,  '#10d32d', 'Vegetation'),   # green
    (5,  '#ffff53', 'Not vegetated'),   # yellow
    (6,  '#0000ff', 'Water'),   # blue
    (7,  '#818181', 'Unclassified'),   # medium gray
    (8,  '#c0c0c0', 'Cloud medium probability'),   # light gray
    (9,  '#f2f2f2', 'cloud high probability'),   # very light gray
    (10, '#bbc5ec', 'Thin cirrus'),   # light blue/purple
    (11, '#53fff9', 'Snow or ice'),   # cyan
]
color_val = [x[0] for x in color_def]
color_hex = [x[1] for x in color_def]
color_txt = [f'{x[0]:2d}: {x[2]}' for x in color_def]
color_lim = (min(color_val), max(color_val) + 1)
bin_edges = color_val + [max(color_val) + 1]
bin_range = (color_val[0] + 0.5, color_val[-1] + 0.5)  # No idea why (0.5,11.5) works and (0,11) or (0,12) do not

# These options manipulate the color map and colorbar to show the categories for this product
options = {
    'title': f'Flag data for: {query["product"]} ({flag_name})',
    'cmap': color_hex,
    'clim': color_lim,
    'color_levels': bin_edges,
    'colorbar': True,
    'width': 800,
    'height': 450,
    'aspect': 'equal',
    'tools': ['hover'],
    'colorbar_opts': {
        'major_label_overrides': dict(zip(color_val, color_txt)),
        'major_label_text_align': 'left',
        'ticker': FixedTicker(ticks=color_val),
    },
}

# Set the Dataset CRS
plot_crs = native_crs
if plot_crs == 'epsg:4326':
    plot_crs = ccrs.PlateCarree()


# Native data and coastline overlay:
# - Comment `crs`, `projection`, `coastline` to plot in native_crs coords
# TODO: Update the axis labels to 'longitude', 'latitude' if `coastline` is used

quality_plot = flag_data.hvplot.image(
    x = 'x', y = 'y',                        # Dataset x,y dimension names
    rasterize = True,                        # Use Datashader
    aggregator = reductions.mode(),          # Datashader selects mode value, requires 'hv.Image'
    precompute = True,                       # Datashader precomputes what it can
    crs = plot_crs,                          # Datset crs
    projection = ccrs.PlateCarree(),         # Output projection (ccrs.PlateCarree() when coastline=True)
    coastline = '10m',                       # Coastline = '10m'/'50m'/'110m'
).options(opts.Image(**options)).hist(bin_range = bin_range)

# display(quality_plot)
# Optional: Change the default time slider to a dropdown list, https://stackoverflow.com/a/54912917
fig = pn.panel(quality_plot, widgets={'time': pn.widgets.Select})  # widget_location='top_left'
display(fig)

# Indonesia, Saleh Bay: every 5th day from 2020-01-05 to 2020-01-25
# Eyeballing with Google Maps suggests the S2 is correct and the coastline is wrong

In [ ]:
# Create Mask layer

good_pixel_flags = [color_def[i][2] for i in [4, 5, 6]]

good_pixel_mask = enum_to_bool(data[flag_name], good_pixel_flags)  # -> DataArray
display(good_pixel_mask)  # Type: bool

## Scaling and nodata

In [ ]:
# Define scaling

# http://step.esa.int/thirdparties/sen2cor/2.8.0/docs/S2-PDGS-MPC-L2A-IODD-V2.8.pdf
# 1 / 10.000: i.e.: Digital Numbers 0 : 10.000, representing radiometric
# reflectance values from 0.0 to 1.0

# Cophub sen2cor S2 products
scale = 0.0001  # div 10000
offset = 0.

# ***---Split---***
# GA S2 products
# scale = 0.0001  # div 10000
# offset = 0.
# flag_name = 'fmask'
# categories =['valid', 'snow', 'water']

In [ ]:
# Select a layer and apply masking and scaling, then persist in dask

layer_name = 'nir'

# Apply valid mask and good pixel mask
layer = data[[layer_name]].where(valid_mask[layer_name] & good_pixel_mask) * scale + offset
layer = layer.persist()

## Visualisation

In [ ]:
# Generate a plot

options = {
    'title': f'{query["product"]}: {layer_name}',
    'width': 800,
    'height': 450,
    'aspect': 'equal',
    'cmap': cc.rainbow,
    'clim': (0, 1),                          # Limit the color range depending on the layer_name
    'colorbar': True,
    'tools': ['hover'],
}

# Set the Dataset CRS
plot_crs = native_crs
if plot_crs == 'epsg:4326':
    plot_crs = ccrs.PlateCarree()


# Native data and coastline overlay:
# - Comment `crs`, `projection`, `coastline` to plot in native_crs coords
# TODO: Update the axis labels to 'longitude', 'latitude' if `coastline` is used

layer_plot = layer.hvplot.image(
    x = 'x', y = 'y',                        # Dataset x,y dimension names
    rasterize = True,                        # Use Datashader
    aggregator = reductions.mean(),          # Datashader selects mean value
    precompute = True,                       # Datashader precomputes what it can
    crs = plot_crs,                        # Dataset crs
    projection = ccrs.PlateCarree(),         # Output projection (use ccrs.PlateCarree() when coastline=True)
    coastline='10m',                         # Coastline = '10m'/'50m'/'110m'
).options(opts.Image(**options)).hist(bin_range = options['clim'])

# display(layer_plot)
# Optional: Change the default time slider to a dropdown list, https://stackoverflow.com/a/54912917
fig = pn.panel(layer_plot, widgets={'time': pn.widgets.Select})  # widget_location='top_left'
display(fig)

# Good image of Sarawak: 2020-01-27 0555

## Optional: Reproject

Use xr_reproject to reproject to UTM data to a new project, e.g. to align with other datasets

See [dea-notebooks/Frequently_used_code/Reprojecting_data.ipynb](#../../../dea-notebooks/Frequently_used_code/Reprojecting_data.ipynb) for additional examples.

In [ ]:
# This example uses the query bounding box and epsg:4326  as the target

target_crs = 'epsg:4326'
target_res = (0.0002, 0.0002)   # approx. 20 m

target = GeoBox.from_geopolygon(box(min_longitude, min_latitude, max_longitude, max_latitude, target_crs), target_res)
layer_geo = xr_reproject(layer, target)

In [ ]:
# Set the Dataset CRS
plot_crs = target_crs
if plot_crs == 'epsg:4326':
    plot_crs = ccrs.PlateCarree()


# Native data and coastline overlay:
# - Comment `crs`, `projection`, `coastline` to plot in native_crs coords
# TODO: Update the axis labels to 'longitude', 'latitude' if `coastline` is used
    
layer_plot = layer_geo.hvplot.image(
    x = 'longitude', y = 'latitude',         # Dataset x,y dimension names
    rasterize = True,                        # Use Datashader
    aggregator = reductions.mean(),          # Datashader selects mean value
    precompute = True,                       # Datashader precomputes what it can
    crs = plot_crs,                          # Dataset crs
    projection = ccrs.PlateCarree(),         # Output projection (use ccrs.PlateCarree() when coastline=True)
    coastline='10m',                         # Coastline = '10m'/'50m'/'110m'
).options(opts.Image(**options)).hist(bin_range = options['clim'])

# display(layer_plot)
# Optional: Change the default time slider to a dropdown list, https://stackoverflow.com/a/54912917
fig = pn.panel(layer_plot, widgets={'time': pn.widgets.Select})  # widget_location='top_left'
display(fig)

## Appendix

Table of Sentinel-2 ARD bands and corresponding product measurement and _alias_ names. This table has been created manually. 

|Band|Resolution (m)|cophub_s2_sr_sen2cor|s2_l2a|ga_s2a_ard_nbar_granule|
|----|----------|--------------------|------|-----------------------|
| B1 | 60 | coastal_aerosol<br>_B01 / B01_60m / band_1_ | B01<br>_band_01 / coastal_aerosol_ | nbar_coastal_aerosol<br>_nbar_band_01 / nbar_B01 / nbar_Band1_ |
| B2 | 10 | blue<br>_B02 / B02_10m / band_2_ | B02<br>_band_02 / blue_ | nbar_blue<br>_nbar_band_02 / nbar_B02 / nbar_Band2_ |
| B3 | 10 | green<br>_B03 / B03_10m / band_3_ | B03<br>_band_03 / green_ | nbar_green<br>_nbar_band_03 / nbar_B03 / nbar_Band3_ |
| B4 | 10 | red<br>_B04 / B04_10m / band_4_ | B04<br>_band_04 / red_ | nbar_red<br>_nbar_band_04 / nbar_B04 / nbar_Band4_ |
| B5 | 20 | red_edge_1<br>_B05 / B05_20m / band_5_ | B05<br>_band_05 / red_edge_1_ | nbar_red_edge_1<br>_nbar_band_05 / nbar_B05 / nbar_Band5_ |
| B6 | 20 | red_edge_2<br>_B06 / B06_20m / band_6_ | B06<br>_band_06 / red_edge_2_ | nbar_red_edge_2<br>_nbar_band_06 / nbar_B06 / nbar_Band6_ |
| B7 | 20 | red_edge_3<br>_B07 / B07_20m / band_7_ | B07<br>_band_07 / red_edge_3_ | nbar_red_edge_3<br>_nbar_band_07 / nbar_B07 / nbar_Band7_ |
| B8 | 10 | nir<br>_B08 / B08_10m / band_8 / nir_1_ | B08<br>_band_08 / nir / nir_1_ | nbar_nir_1<br>_nbar_band_08 / nbar_B08 / nbar_Band8_ |
| B8a | 20 | nir_narrow<br>_B8A / B8A_20m / band_8a / nir_2_ | B8A<br>_band_8a / nir_narrow / nir_2_ | nbar_nir_2<br>_nbar_band_8A / nbar_B8A / nbar_Band8A_ |
| B9 | 60 | water_vapour<br>_B09 / B09_60m / band_09_ | B09<br>_band_09 / water_vapour_ | - |
| B10 | 60 | - | - | - |
| B11 | 20 | swir1<br>_B11 / B11_20m / band_11 / swir_1 / swir_16_ | B11<br>_band_11 / swir_1 / swir_16_ | nbar_swir_2<br>_nbar_band_11 / nbar_B11 / nbar_Band11_ |
| B12 | 20 | swir2<br>_B12 / B12_20m / band_12 / swir_2 / swir_22_ | B12<br>_band_12 / swir_2, / swir_22_ | nbar_swir_3<br>_nbar_band_12 / nbar_B12 / nbar_Band12_ |
| - | 20 | quality<br>_SCL / SCL_20m / mask / qa_ | SCL<br>_mask / qa_ | - |
| - | - | - | - | fmask<br>_mask / Fmask_ |
| - | 20 | aerosol_optical_thickness<br>_AOT / AOT_20m_ | AOT<br>_aerosol_optical_thickness_ | |
| - | 20 | water_vapour_retrieval<br>_WVP / WVP_20m / scene_average_water_vapour_ | WVP<br>_scene_average_water_vapour_ | |
| - | - | - | - | azimuthal_exiting |
| - | - | - | - | azimuthal_incident |
| - | - | - | - | exiting |
| - | - | - | - | incident |
| - | - | - | - | relative_azimuth |
| - | - | - | - | relative_slope |
| - | - | - | - | satellite_azimuth |
| - | - | - | - | satellite_view |
| - | - | - | - | solar_azimuth |
| - | - | - | - | solar_zenith |
| - | - | - | - | terrain_shadow |
| - | - | - | - | nbar_contiguity |

#### EC Copernicus Data Hub

https://scihub.copernicus.eu

EC Copernicus Data Hub is the primary source of S2 L1B and L2A (Sen2cor) data. L2A (Sen2cor) is available globally from December 2018 onwards.

Atmospheric correction - Sen2cor
- Detailed summary of bands and thresholds for classification: https://sentinel.esa.int/web/sentinel/technical-guides/sentinel-2-msi/level-2a/algorithm
- Details of Sen2cor paramaters: https://sentinels.copernicus.eu/documents/247904/685211/Sen2-Cor-L2A-Input-Output-Data-Definition-Document

Atmospheric correction - Any other method, including Sen2cor prior to December 2018
- EASI can inject just about any preprocessing routine into our data preparation pipeline. That is, we would download S2 L1B and apply a preprocessing routine (an atmospheric correction method). This would create a new product name like ```cophub_s2_[type]_[routine]```.

#### Sentinel-2 Cloud-Optimized GeoTIFFs

https://registry.opendata.aws/sentinel-2-l2a-cogs/

Open and free S2 L2a (Sen2cor) COGs. Same time range as from CopHub (December 2018 onwards).

TODO: Check the latency between CopHub and AWS.

#### Geoscience Australia NBAR-corrected for Australia

_Adapted from https://github.com/GeoscienceAustralia/dea-notebooks/blob/develop/DEA_datasets/Sentinel_2.ipyn

Digital Earth Australia (DEA) applies corrections to Sentinel-2 satellite images to arrive at a [surface reflectance](https://cmi.ga.gov.au/ga_s2_m_nbart_1) product (see the [introduction to Digtial Earth Australia](../Beginners_guide/02_DEA.ipynb) section for more information on the surface reflectance corrections).
Surface reflectance provides standardised optical datasets by using robust physical models to correct for variations in image radiance values due to atmospheric properties, as well as sun and sensor geometry.
The resulting stack of surface reflectance grids are consistent over space and time, which is instrumental in identifying and quantifying environmental change.

DEA provides two Sentinel-2 surface reflectance products:

- **Sentinel-2 Definitive** (e.g. `s2a_ard_granule`): These products represent the 'definitive' source of high quality Sentinel-2 surface reflectance data, and are available from the beginning of the Sentinel-2 archive up to a delay of several weeks. 

- **Sentinel-2 Near Real Time** (e.g. `s2a_nrt_granule`): These products are processed with best-available ancillary information and provided as a rolling 90 day archive of imagery which is typically available to load within approximately ~24 hours of a satellite overpass.

Both Sentinel-2 Definitive and Sentinel-2 Near Real Time products contain data processed to two surface reflectance corrections:

- **NBAR** (e.g. `nbar_green`): NBAR stands for Nadir-corrected BRDF Adjusted Reflectance, where BRDF stands for Bidirectional reflectance distribution function.
    The approach involves atmospheric correction to compute surface-leaving radiance and bi-directional reflectance modelling to remove the effects of topography and angular variation in reflectance.

- **NBAR-T** (e.g. `nbart_green`): Surface reflectance NBAR-T includes the terrain illumination reflectance correction and has the same features of NBAR, along with some additional features.